# Live Trading Demo

This notebook runs MT5 first, then IBKR, and prints connection/runtime status to stdout.


In [1]:
import os
import sys
import asyncio
import threading
from pathlib import Path

import yaml

sys.path.append(os.path.abspath("../.."))

from nautilus_trader.config import TradingNodeConfig, LiveExecEngineConfig, LoggingConfig
from nautilus_trader.live.node import TradingNode
from nautilus_trader.model.identifiers import ClientId, InstrumentId
from trader import OneMinuteBuyHoldStrategy, OneMinuteBuyHoldConfig

accounts_path = Path("../../config/accounts.yaml").resolve()
accounts = yaml.safe_load(accounts_path.read_text(encoding="utf-8"))
print(f"Using accounts config: {accounts_path}")

project_root = accounts_path.parent.parent
log_dir = project_root / "tests" / "notebooks"
log_dir.mkdir(parents=True, exist_ok=True)

# Nautilus logging is process-global, so both runs share one runtime log file.
live_log_file_name = "nt_live_demo"
live_log_path = log_dir / f"{live_log_file_name}.log"
print(f"Shared Nautilus runtime log file: {live_log_path}")

# Per-run snapshots (copied from shared runtime log after each run).
mt5_log_path = log_dir / "nt_mt5.log"
ib_log_path = log_dir / "nt_ibkr.log"
print(f"MT5 snapshot log file: {mt5_log_path}")
print(f"IBKR snapshot log file: {ib_log_path}")

mt5_node_logging = LoggingConfig(
    log_level="INFO",
    log_level_file="INFO",
    log_directory=str(log_dir),
    log_file_name=live_log_file_name,
    clear_log_file=True,
    log_colors=False,
)

ib_node_logging = LoggingConfig(
    log_level="INFO",
    log_level_file="INFO",
    log_directory=str(log_dir),
    log_file_name=live_log_file_name,
    clear_log_file=True,
    log_colors=False,
)


Using accounts config: S:\Github\Trading\config\accounts.yaml
Shared Nautilus runtime log file: S:\Github\Trading\tests\notebooks\nt_live_demo.log
MT5 snapshot log file: S:\Github\Trading\tests\notebooks\nt_mt5.log
IBKR snapshot log file: S:\Github\Trading\tests\notebooks\nt_ibkr.log


## MT5 Run

Loads instruments into cache, starts the node in a background thread, prints health, then stops.


In [2]:
from trader.adapters.metatrader import (
    MetaTrader5DataClientConfig,
    MetaTrader5ExecClientConfig,
    MetaTrader5LiveDataClientFactory,
    MetaTrader5LiveExecClientFactory,
)

mt5_cfg = accounts["venues"]["MT5"]

mt5_data_config = MetaTrader5DataClientConfig(
    mt5_login=mt5_cfg["mt5_login"],
    mt5_password=mt5_cfg["mt5_password"],
    mt5_server=mt5_cfg["mt5_server"],
    mt5_path=mt5_cfg["mt5_path"],
    bar_seconds=5,
    poll_interval=0.5,
)

mt5_exec_config = MetaTrader5ExecClientConfig(
    mt5_login=mt5_cfg["mt5_login"],
    mt5_password=mt5_cfg["mt5_password"],
    mt5_server=mt5_cfg["mt5_server"],
    mt5_path=mt5_cfg["mt5_path"],
    mt5_deviation=20,
)

mt5_strategy = OneMinuteBuyHoldStrategy(
    OneMinuteBuyHoldConfig(
        instrument_id="EURUSD.MT5",
        bar_type="EURUSD.MT5-5-SECOND-MID-EXTERNAL",
        trade_size=0.1,
        hold_seconds=5,
    )
)

mt5_node = TradingNode(
    TradingNodeConfig(
        data_clients={"MT5": mt5_data_config},
        exec_clients={"MT5": mt5_exec_config},
        exec_engine=LiveExecEngineConfig(reconciliation=False),
        logging=mt5_node_logging,
    )
)
mt5_node.add_data_client_factory("MT5", MetaTrader5LiveDataClientFactory)
mt5_node.add_exec_client_factory("MT5", MetaTrader5LiveExecClientFactory)
mt5_node.trader.add_strategy(mt5_strategy)
mt5_node.build()

# Manually load instruments into cache before strategy start.
mt5_dc = mt5_node.kernel.data_engine._clients[ClientId("MT5")]
mt5_provider = mt5_dc._instrument_provider
await mt5_provider.load_all_async()
for inst in mt5_provider._instruments.values():
    mt5_node.cache.add_instrument(inst)

cached = mt5_node.cache.instrument(InstrumentId.from_str("XAUUSD.MT5"))
print("MT5 instrument cached:", cached is not None)

mt5_node.run()

for i in range(5):
    await asyncio.sleep(5)
    print(
        f"MT5 t={5 * (i + 1)}s connected data={mt5_node.kernel.data_engine.check_connected()} exec={mt5_node.kernel.exec_engine.check_connected()}"
    )

await mt5_node.stop_async()
while mt5_node.is_running():
    await asyncio.sleep(0.1)
mt5_log_path.write_bytes(live_log_path.read_bytes())
print(f"MT5 run complete. Snapshot saved to: {mt5_log_path}")


MT5 instrument cached: True
MT5 t=5s connected data=True exec=True
MT5 t=10s connected data=True exec=True
MT5 t=15s connected data=True exec=True
MT5 t=20s connected data=True exec=True
MT5 t=25s connected data=True exec=True
MT5 run complete. Snapshot saved to: S:\Github\Trading\tests\notebooks\nt_mt5.log


## IBKR Run

Attempts a live IBKR run and prints status/errors to stdout.


In [2]:
from nautilus_trader.adapters.interactive_brokers.factories import (
    InteractiveBrokersLiveDataClientFactory,
    InteractiveBrokersLiveExecClientFactory,
)
from ibapi.common import MarketDataTypeEnum as IBMarketDataTypeEnum
from nautilus_trader.adapters.interactive_brokers.config import SymbologyMethod
from trader.adapters.ibkr import (
    ibkr_data_config,
    ibkr_exec_config,
    ibkr_instrument_config,
)

ib_cfg = accounts["venues"].get("IDEALPRO", {})
ib_host = ib_cfg.get("host", "127.0.0.1")
ib_port = int(ib_cfg.get("port", 7497))
ib_client_id = int(ib_cfg.get("client_id", 1))
ib_account = ib_cfg.get("account", "")

ib_instr_cfg = ibkr_instrument_config(
    load_ids=frozenset({InstrumentId.from_str("AAPL.NYSE")}),
    symbology_method=SymbologyMethod.IB_SIMPLIFIED,
)

ib_data = ibkr_data_config(
    host=ib_host,
    port=ib_port,
    client_id=ib_client_id,
    market_data_type=IBMarketDataTypeEnum.DELAYED_FROZEN,
    instrument_provider=ib_instr_cfg,
)
ib_exec = ibkr_exec_config(
    host=ib_host,
    port=ib_port,
    client_id=ib_client_id,
    account=ib_account,
    instrument_provider=ib_instr_cfg,
)

ib_strategy = OneMinuteBuyHoldStrategy(
    OneMinuteBuyHoldConfig(
        instrument_id="AAPL.NYSE",
        bar_type="AAPL.NYSE-15-SECOND-LAST-EXTERNAL",
        trade_size=0.01,
        hold_seconds=5,
    )
)

ib_node = TradingNode(
    TradingNodeConfig(
        data_clients={"IB": ib_data},
        exec_clients={"IB": ib_exec},
        exec_engine=LiveExecEngineConfig(reconciliation=False),
        logging=ib_node_logging,
    )
)
ib_node.add_data_client_factory("IB", InteractiveBrokersLiveDataClientFactory)
ib_node.add_exec_client_factory("IB", InteractiveBrokersLiveExecClientFactory)
ib_node.trader.add_strategy(ib_strategy)
ib_node.build()

ib_node.run()

for i in range(12):
    await asyncio.sleep(5)
    print(
        f"IBKR t={5 * (i + 1)}s connected data={ib_node.kernel.data_engine.check_connected()} exec={ib_node.kernel.exec_engine.check_connected()}"
    )

await ib_node.stop_async()
while ib_node.is_running():
    await asyncio.sleep(0.1)
ib_log_path.write_bytes(live_log_path.read_bytes())
ib_log_text = ib_log_path.read_text(encoding="utf-8", errors="ignore")
print(f"IBKR submit count: {ib_log_text.count('SubmitOrder')}")
if "code: 420" in ib_log_text:
    print("IBKR market-data entitlement error (420). Enable US equity market data or switch instrument.")
print(f"IBKR run complete. Snapshot saved to: {ib_log_path}")


IBKR t=5s connected data=True exec=True
IBKR t=10s connected data=True exec=True
IBKR t=15s connected data=True exec=True
IBKR t=20s connected data=True exec=True
IBKR t=25s connected data=True exec=True
IBKR t=30s connected data=True exec=True
IBKR t=35s connected data=True exec=True
IBKR t=40s connected data=True exec=True
IBKR t=45s connected data=True exec=True
IBKR t=50s connected data=True exec=True
IBKR t=55s connected data=True exec=True
IBKR t=60s connected data=True exec=True
IBKR submit count: 0
IBKR run complete. Snapshot saved to: S:\Github\Trading\tests\notebooks\nt_ibkr.log
